In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import polars.selectors as cs
import seaborn as sns
from plotly.subplots import make_subplots

pl.Config.set_tbl_rows(25)

# Movie Metadata Analysis
## Extract
1. Read the CSV into a dataframe.
    - `mm_raw` - raw movie metadata dataframe
    - `mm` - transformed movie metadata dataframe
    - `Released_Year` is mapped to a string because there is at least 1 value in that column that cannot be inferred as an integer
    - this is investigated in additional cells below
2. Remove columns that will not be used in the analysis:
    - `Poster_Link`
    - `Overview`
    - `Certificate`
3. Convert all column names to lowercase for uniformity.
4. Adjust the max allowed length of strings in outputs to easily see the full titles of movies
    - the movie title is treated as the priority for viewing string output in the dataframe

In [ ]:
movie_metadata_path: Path = Path.cwd() / "imdb_top_1000.csv"

mm_raw: DataFrame = pl.read_csv(
    source=movie_metadata_path, schema_overrides={"Released_Year": pl.String}
)

mm: DataFrame = (
    mm_raw
    .select(pl.col("*").exclude("Poster_Link", "Overview", "Certificate"))
    .rename(lambda col_name: col_name.lower())
)

max_title_len: int = mm["series_title"].str.len_bytes().max()
pl.Config.set_fmt_str_lengths(max_title_len)

print(f"full dataframe shape: {mm.shape}")
mm.head()

## Explore
### Columns Per Data Type

In [ ]:
# First check what data types are in the dataframe.
pl.Series(mm.dtypes).value_counts(sort=True)

#### Integer Columns

In [ ]:
mm.select(cs.integer()).columns

#### String Columns

In [ ]:
mm.select(cs.string()).columns

`released_year`, `runtime`, and `gross` should all be integer columns.

#### Float Columns

In [ ]:
mm.select(cs.float()).columns

### Find the `released_year` Values That Prevent Casting to Integer Column

In [ ]:
# Cast to integer by supressing the error that results. Then find the null values.
mm.filter(pl.col("released_year").cast(pl.Int64, strict=False).is_null())

The only problem row preventing casting is the movie `"Apollo 13"`. The release year for this movie is `1995` and will be updated in the **Column Transformations** section below.

### Missing/Null Values

In [ ]:
mm.null_count()

Null values are only found in the `gross` and `meta_score` columns. This is not really an issue therefore no action will be taken for null values.

### Duplicates
#### `series_title` Duplicates

In [ ]:
mm.filter(pl.col("series_title").is_duplicated())

There are two movies that share the same title but are clearly different movies. No action is needed.

#### Combination of `series_title` and `released_year` Duplicates

In [ ]:
# `pl.struct` is used to tie the two columns together in one object.
mm.filter(pl.struct("series_title", "released_year").is_duplicated())

There are no duplicates when `series_title` and `released_year` are used in combination as a unique identifier for each row. Although not the goal of the project, if a database was constructed, `series_title` with `released_year` could be used as a composite primary key.

## Transformations
### Column Reordering

In [ ]:
# A more logical ordering.
col_order = [
    "series_title",
    "released_year",
    "genre",
    "director",
    "star1",
    "star2",
    "star3",
    "star4",
    "runtime",
    "gross",
    "meta_score",
    "imdb_rating",
    "no_of_votes",
]

mm = mm.select(col_order)
mm.head()

### Column Transformations
1. `released_year`
    - There is one value that is preventing the column from being cast as an integer column. The movie is `"Apollo 13"`; the `released_year` value is `"PG"` and should be changed to `"1995"`. Then the column can be cast to an integer column. `pl.lit()` is needed because polars would otherwise try to look for a column named `"1995"`. Now the column can be cast to an integer type.
1. `genre`
    - The values need to be split on ", ". The split operation will cast the column as a list type.
1. `runtime`
    - The characters " min" need to be removed. Then cast to an integer type.
1. `gross`
    - All of the "," characters need to be removed. Then cast to an integer type.

In [ ]:
mm = mm.with_columns(
    # Convert `released_year` to integer column.
    pl.when(pl.col("released_year") == "PG")
    .then(pl.lit("1995"))
    .otherwise(pl.col("released_year"))
    .cast(pl.Int16)
    .alias("released_year"),
    # Convert `genre` to a list Column.
    pl.col("genre").str.split(", "),
    # Address the `runtime` column.
    pl.col("runtime").str.strip_chars_end(characters=" min").cast(pl.Int16),
    # Address the `gross` column.
    pl.col("gross").str.replace_all(pattern=",", value="", literal=True).cast(pl.Int32),
)
mm.head()

### Distributions of Numeric Columns

In [ ]:
# Select only the numeric columns.
numeric_data = mm.select(cs.numeric())

# Create a 2x3 subplot figure.
fig = make_subplots(
    rows=2,
    cols=3,
    subplot_titles=numeric_data.columns,
    horizontal_spacing=0.1,
    vertical_spacing=0.09,
)

# Create a "flat" list of subplot references in order to assign each column's boxplot in
# the for loop below.
subplot_refs = [(row, col) for row in range(1, 3) for col in range(1, 4)]

for (row, col), col_name in zip(subplot_refs, numeric_data.columns):
    # Create box plot and add to figure.
    box_plot = px.box(numeric_data, y=col_name)
    fig.add_trace(box_plot.data[0], row=row, col=col)

fig.update_layout(
    height=800,
    width=800,
    title={
        "text": "Numeric Column Distributions",
        "y": 0.965,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    yaxis3_type="log",
    yaxis6_type="log",
    showlegend=False,
    margin=dict(t=85, l=30, r=30, b=30),
)

fig.show()

In [ ]:
mm.describe()

### Handling Genres

In [ ]:
mm.select(pl.col("genre").list.explode().unique())

There are 21 unique genres. For analysis operations, the genres could be multi-hot encoded across 21 new columns. Instead, the functionality of the polars `List` type column will be used for the operations involving the genres.

## Analysis

### Top Directors Average IMDB Rating
Find the 3 directors with the most movies. What is the average imdb score for each?

In [ ]:
(
    mm
    .group_by("director")
    .agg(
        pl.len().alias("movie_count"),
        pl.mean("imdb_rating").round(2).alias("avg_imdb_rating"),
    )
    .sort(by="movie_count", descending=True)
    .head(3)
)

### Actor Roles
Find actors with the most leading roles (`star1`). Find the actors with the most roles.
#### Actors with the most leading roles (`star1`)

In [ ]:
(
    mm
    .group_by("star1")
    .len()
    .sort(by="len", descending=True)
    .head(10)
    .select(
        pl.col("star1").alias("actor"),
        pl.col("len").alias("leading_roles"),
    )
)

#### Actors with the most roles

In [ ]:
(
    mm
    .select(
        pl.col("star1"),
        pl.col("star2"),
        pl.col("star3"),
        pl.col("star4"),
    )
    .melt()
    .group_by("value")
    .len()
    .sort(by="len", descending=True)
    .head(10)
    .select(
        pl.col("value").alias("actor"),
        pl.col("len").alias("roles"),
    )
)

### Director/Actor Pairings
For directors Steven Spielberg and Martin Scorsese, which actors have they worked with the most.
#### Steven Spielberg

In [ ]:
(
    mm
    .filter(pl.col("director") == "Steven Spielberg")
    .select(
        pl.col("star1"),
        pl.col("star2"),
        pl.col("star3"),
        pl.col("star4"),
    )
    .melt()
    .group_by("value")
    .len()
    .sort(by="len", descending=True)
    .head(3)
    .select(
        pl.col("value").alias("actor"),
        pl.col("len").alias("worked_with_spielberg"),
    )
)

#### Martin Scorsese

In [ ]:
(
    mm
    .filter(pl.col("director") == "Martin Scorsese")
    .select(
        pl.col("star1"),
        pl.col("star2"),
        pl.col("star3"),
        pl.col("star4"),
    )
    .melt()
    .group_by("value")
    .len()
    .sort(by="len", descending=True)
    .head(3)
    .select(
        pl.col("value").alias("actor"),
        pl.col("len").alias("worked_with_scorsese"),
    )
)

### Highest Rated Movie 2006-2016
Find the highest rated movie in each year from 2006-2016.

In [ ]:
(
    mm
    .filter(pl.col("released_year").is_between(2006, 2016))
    .group_by("released_year")
    .agg(
        pl.col('series_title').filter(pl.col('imdb_rating') == pl.col('imdb_rating').max()).alias("highest_rated_movies"),
        pl.max("imdb_rating"),
    )
    .sort(by="released_year")
)

### Plots
#### Relationship between `runtime` and `gross`

In [ ]:
# Filter out movies that don't have a `gross` value.
filtered_mm = mm.filter(pl.col("gross").is_not_null())

# Add 2 new columns representing the decade of release year and the imdb rating group.
# \u2264 and \u2265 are less than or equal to and greater than or equal to respectively.
filtered_mm = (
    filtered_mm
    .with_columns(
        ((pl.col("released_year") // 10) * 10).alias("decade"),
        pl.col("imdb_rating")
        .cut(
            breaks=[8.0, 9.0],
            labels=["rating < 8", "8 \u2264 rating < 9", "rating \u2265 9"],
            left_closed=True,
        )
        .alias("imdb_rating_group"),
    )
    .sort(by="decade")
)

# fig = px.scatter(
#     filtered_mm,
#     x="runtime",
#     y="gross",
#     log_y=True,
#     color="decade",
#     title="Gross Earnings vs. Runtime",
#     labels={"runtime": "Runtime [min]", "gross": "Gross [$]"},
#     custom_data="series_title",
# )

fig = px.scatter(
    filtered_mm,
    x="runtime",
    y="gross",
    log_y=True,
    color="imdb_rating_group",
    facet_col="decade",
    facet_col_wrap=5,
    title="Gross Earnings vs. Runtime",
    labels={"runtime": "Runtime [min]", "gross": "Gross [$]", "decade": "Decade"},
    custom_data="series_title",
    opacity=0.6,
)

fig.update_layout(
    height=400,
    width=1200,
    title={
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    margin=dict(t=65, l=30, r=30, b=30),
)

fig.update_traces(
    hovertemplate="<b>Movie:</b> %{customdata}<br>"
                + "<b>Runtime:</b> %{x} minutes<br>"
                + "<b>Gross:</b> $%{y:,.0f}<extra></extra>",
)

fig.show()

#### Relationship between `imdb_rating` and `gross`

In [ ]:
# Filter out movies that don't have a `gross` value.
filtered_mm = mm.filter(pl.col("gross").is_not_null())

# Add 2 new columns representing the decade of release year and the imdb rating group.
filtered_mm = (
    filtered_mm
    .with_columns(
        ((pl.col("released_year") // 10) * 10).alias("decade"),
        pl.col("imdb_rating")
        .cut(
            breaks=[8.0, 9.0],
            labels=["rating < 8", "8 <= rating < 9", "rating >= 9"],
            left_closed=True,
        )
        .alias("imdb_rating_group"),
    )
    .sort(by="decade")
)

fig = px.histogram(
    mm,
    x="imdb_rating",
    y="gross",
    nbins=20,
    histfunc="avg",
    labels={"imdb_rating": "IMDB Rating", "gross": "Gross Earnings"},
    title="Movie Gross Earnings by IMDB Rating",
    custom_data=["series_title", "released_year"],
)

# fig = px.scatter(
#     filtered_mm,
#     x="imdb_rating",
#     y="gross",
#     log_y=True,
#     color="decade",
#     title="Gross Earnings vs. IMDB Rating",
#     labels={"imdb_rating": "IMDB Rating", "gross": "Gross [$]"},
#     custom_data=["series_title", "released_year"],
#     color_continuous_scale="Plasma_r",
# )

fig.update_layout(
    # height=400,
    # width=900,
    title={
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    # margin=dict(t=65, l=30, r=30, b=30),
    # coloraxis_colorbar={"title": "Decade"},
)

fig.update_traces(
    hovertemplate="<b>Movie:</b> %{customdata[0]}<br>"
                + "<b>Release Year:</b> %{customdata[1]}<br>"
                + "<b>IMDB Rating:</b> %{x}<br>"
                + "<b>Gross:</b> $%{y:,.0f}
                <extra></extra>",
)

fig.show()

### Genres
Average `imdb_Rating` rating per genre.

In [ ]:
(
    mm
    .select(
        pl.col("genre"),
        pl.col("imdb_rating")
    )
    .explode("genre")
    .group_by("genre")
    .agg(
        pl.col("imdb_rating").mean().round(2).alias("avg_imdb_rating"),
        pl.len().alias("movie_count"),
    )
    .sort(by="movie_count", descending=True)
)